### Fake dataset

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision as tv

In [15]:
tfms = tv.transforms.Compose([tv.transforms.ToTensor(),
                             tv.transforms.Normalize((0.5,0.5,0.5),(0.5,.5,.5))])
ds = tv.datasets.FakeData(transform=tfms)
dl = torch.utils.data.DataLoader(ds,batch_size=100,shuffle=True)

net = tv.models.resnet18(pretrained=True)

In [16]:
torch.set_grad_enabled(False) # disable grad 

net.fc = nn.Linear(in_features=512, out_features=10, bias=True)

In [17]:
for i in net.parameters():
    i.requires_grad = False
net.fc.weight.requires_grad = True
net.fc.bias.requiers_grad = True

## Single Batch Predictions

In [23]:
x,y = next(iter(dl))

one batch prediction

In [24]:
predict = net(x)
predict = F.softmax(predict,dim=1)
predict.argmax(dim=1)

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 3, 1, 1, 3, 1, 1, 3, 1, 1, 3,
        1, 1, 1, 1, 1, 1, 3, 1, 3, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1,
        1, 1, 1, 1, 3, 1, 1, 1, 3, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1,
        1, 1, 3, 1])

To compare with y use `eq` function

In [28]:
predict.argmax(dim=1).eq(y).sum()

tensor(11)

Define a prediction function:

In [34]:
def num_correct(predict,label):
    correct = predict.argmax(dim=1).eq(label).sum()
    return correct.item()

In [35]:
num_correct(predict,y)

11

In [47]:
predict.max(dim=1).values


tensor([0.4256, 0.2751, 0.4044, 0.2591, 0.3352, 0.2602, 0.3783, 0.5520, 0.2984,
        0.2858, 0.3104, 0.4339, 0.3315, 0.2444, 0.2064, 0.2362, 0.3658, 0.2935,
        0.3740, 0.3841, 0.3118, 0.3776, 0.1776, 0.3917, 0.2164, 0.4122, 0.2490,
        0.3091, 0.2659, 0.2879, 0.3794, 0.2401, 0.4450, 0.3219, 0.2243, 0.2090,
        0.3554, 0.3440, 0.3170, 0.2257, 0.2632, 0.2089, 0.3746, 0.2181, 0.2153,
        0.3278, 0.2771, 0.3504, 0.2965, 0.2322, 0.3615, 0.2273, 0.2188, 0.2409,
        0.3122, 0.3790, 0.2078, 0.2924, 0.2624, 0.2218, 0.3648, 0.3025, 0.3176,
        0.2622, 0.3791, 0.2606, 0.2921, 0.3260, 0.3552, 0.3873, 0.2705, 0.3419,
        0.4435, 0.2816, 0.2805, 0.3282, 0.3385, 0.4122, 0.3644, 0.2839, 0.2487,
        0.3154, 0.4441, 0.3942, 0.2153, 0.2430, 0.1983, 0.3332, 0.2490, 0.3064,
        0.2298, 0.2281, 0.2056, 0.3268, 0.3742, 0.2350, 0.2080, 0.3700, 0.2086,
        0.2535])

## Get Predictions for the entire training set

#### 1. Loop through the dataloader

In [48]:
@torch.no_grad()
def get_all_preds(model, loader):
    all_preds = torch.tensor([])
    targets
    for batch in loader:
        images, labels = batch

        preds = model(images)
        all_preds = torch.cat(
            (all_preds, preds)
            ,dim=0
        )
    return all_preds

In [49]:
get_all_preds(net,dl)

tensor([[ 0.6681,  0.7576, -0.0761,  ...,  0.2774,  0.3135,  0.1969],
        [ 0.5639,  1.4598, -0.2292,  ...,  0.1145,  0.0816, -0.1499],
        [ 0.6646,  1.8453, -0.3320,  ...,  0.6000,  0.0594,  0.4885],
        ...,
        [ 0.6180,  1.6585, -0.2911,  ...,  0.3005,  0.3355,  0.4243],
        [ 0.8215,  1.7869, -0.0797,  ...,  0.4893,  0.1407,  0.2800],
        [ 0.3575,  1.4516, -0.0435,  ...,  0.5190,  0.0772,  0.0443]])

#### 2.Create a dataloader where batch size equals sample size

In [50]:
with torch.no_grad():
    dll = torch.utils.data.DataLoader(ds,batch_size=len(ds))
    all_pred = get_all_preds(net,dll)

In [51]:
all_pred.shape

torch.Size([1000, 10])

Note: most datasets have the attribute `.targets`. Which means we don't need to loop through all pathers to get them, but in case it's missing, you can extract it by adding `label = torch.tensor([])` to `get_all_preds`